In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torchmetrics.classification import MulticlassAccuracy

In [2]:
no_epochs = 100
learning_rate = 0.001
batch_size = 256

acc_function = MulticlassAccuracy(num_classes=102, average='micro')
loss_fn = nn.CrossEntropyLoss()

In [3]:
transformation = transforms.Compose([
    models.VGG16_BN_Weights.IMAGENET1K_V1.transforms()
])

flowers_train = datasets.Flowers102(root='./data', split='train', download=True, transform=transformation)
flowers_test = datasets.Flowers102(root='./data', split='test', download=True, transform=transformation)
flowers_val = datasets.Flowers102(root='./data', split='val', download=True, transform=transformation)

In [4]:
def get_data_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(flowers_train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(flowers_test, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(flowers_val, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader, val_loader

In [5]:
def train(model, optimizer, dataloader, loss_fn=loss_fn):
    loss_value = 0
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss_value += loss.item()
        loss.backward()
        optimizer.step()
    return loss_value / len(dataloader)

def test_eval(model, dataloader, loss_fn=loss_fn):
    loss = 0
    acc = 0
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            loss += loss_fn(outputs, labels).item()
            acc += acc_function(outputs, labels).item()
    acc /= len(dataloader)
    loss /= len(dataloader)
    print(f'Accuracy: {acc}, Loss: {loss}')
    return acc, loss

def train_eval_loop(model, train_dataloader, val_dataloader, no_epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr = [], [], [], []
    for i in range(no_epochs):
        print(f'Epoch {i+1}')
        train_loss, train_acc = train(model, optimizer, train_dataloader)
        eval_loss, eval_acc = test_eval(model, val_dataloader)
        print(f'Train Accuracy: {train_acc}, Train Loss: {train_loss}, Eval Accuracy: {eval_acc}, Eval Loss: {eval_loss}')
        train_loss_arr.append(train_loss)
        train_acc_arr.append(train_acc)
        eval_loss_arr.append(eval_loss)
        eval_acc_arr.append(eval_acc)
    return train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr
        

In [6]:
def create_model():
    model = models.vgg16_bn(pretrained=True)
    new_classifier_head = nn.Sequential(
        nn.Linear(25088, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 102),
        nn.Softmax(dim=1)
    )
    model.classifier = new_classifier_head
    print(model)
    return model

In [7]:
model = create_model()
train_data_loader, test_data_loader, val_data_loader = get_data_loader(batch_size)
train_acc, train_loss, eval_acc, eval_loss = train_eval_loop(model, train_data_loader, val_data_loader, no_epochs=no_epochs)

/Users/elliot/anaconda3/envs/CZ4042_1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/elliot/anaconda3/envs/CZ4042_1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256